In [8]:
library(ArchR)

In [9]:
setwd("/project/scATAC_analysis/scATAcat_review/preprocessing/Granja2019/BMMC_D6T1/05_get_peaks_and_genesScore_matrix_ArchR/")

In [10]:
set.seed(2022)
addArchRThreads(threads = 1) 
addArchRGenome("hg38")

Setting default number of Parallel threads to 1.

Setting default genome to Hg38.



In [11]:
inputFiles = "/project/scATAC_analysis/scATAcat_review/data/Granja2019/BMMC_D6T1/01_prepare_fragments_file/final_fragment_files/GSM4138889_scATAC_BMMC_D6T1_liftOverhg38_no_chrMt_sorted_fragments.tsv.gz"

In [12]:
ArrowFiles <- createArrowFiles(
  inputFiles = inputFiles,
  sampleNames = "BMMC_D6T1",
  minTSS = 4, #Dont set this too high because you can always increase later
  minFrags = 1000, 
  addTileMat = TRUE,
  addGeneScoreMat = TRUE
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

ArchR logging to : ArchRLogs/ArchR-createArrows-b94d6db1d353-Date-2024-05-12_Time-22-00-19.log
If there is an issue, please report to github with logFile!

Cleaning Temporary Files

2024-05-12 22:00:22 : Batch Execution w/ safelapply!, 0 mins elapsed.

(BMMC_D6T1 : 1 of 1) Determining Arrow Method to use!

2024-05-12 22:00:22 : (BMMC_D6T1 : 1 of 1) Reading In Fragments from inputFiles (readMethod = tabix), 0.004 mins elapsed.

2024-05-12 22:00:22 : (BMMC_D6T1 : 1 of 1) Tabix Bed To Temporary File, 0.005 mins elapsed.

Warning message in sprintf("%s Reading TabixFile %s Percent", prefix, round(100 * :
“one argument not used by format '%s Reading TabixFile %s Percent'”
2024-05-12 22:04:07 : (BMMC_D6T1 : 1 of 1) Reading TabixFile 8 Percent, 3.748 mins elapsed.

Warning message in sprintf("%s Reading TabixFile %s Percent", prefix, round(100 * :
“one argument not used by format '%s Reading 

In [13]:
proj1 <- ArchRProject(
  ArrowFiles = ArrowFiles, 
  outputDirectory = "BMMC_D6T1",
  copyArrows = TRUE #This is recommened so that you maintain an unaltered copy for later usage.
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

Validating Arrows...

Getting SampleNames...

1 


Copying ArrowFiles to Ouptut Directory! If you want to save disk space set copyArrows = FALSE

1 


Getting Cell Metadata...

1 


Merging Cell Metadata...

Initializing ArchRProject...


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--'

In [14]:
getAvailableMatrices(proj1)

[1] "GeneScoreMatrix" "TileMatrix"

In [16]:
dim(getMatrixFromProject(proj1, useMatrix='GeneScoreMatrix'))

ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-b94d14e36d76-Date-2024-05-12_Time-23-12-27.log
If there is an issue, please report to github with logFile!

2024-05-12 23:13:07 : Organizing colData, 0.665 mins elapsed.

2024-05-12 23:13:07 : Organizing rowData, 0.666 mins elapsed.

2024-05-12 23:13:07 : Organizing rowRanges, 0.666 mins elapsed.

2024-05-12 23:13:07 : Organizing Assays (1 of 1), 0.667 mins elapsed.

2024-05-12 23:13:07 : Constructing SummarizedExperiment, 0.668 mins elapsed.

2024-05-12 23:13:08 : Finished Matrix Creation, 0.686 mins elapsed.



[1] 24919 13745

In [17]:
out_path = "/project/scATAC_analysis/scATAcat_review/data/Granja2019/BMMC_D6T1/05_peaks_and_genesScore_matrix_ArchR/input_for_EpiAnno/" # where to save resulting matrices | with / at the end
threads_=1 # how many threads to use

proj1 <- addIterativeLSI(
    ArchRProj = proj1,
    useMatrix = "TileMatrix", 
    name = "IterativeLSI", 
    iterations = 2, 
    clusterParams = list( #See Seurat::FindClusters
        resolution = c(0.2), 
        sampleCells = 10000, 
        n.start = 10
    ), 
    varFeatures = 25000, 
    dimsToUse = 1:30
)

proj1 <- addClusters(
    input = proj1,
    reducedDims = "IterativeLSI",
    method = "Seurat",
    name = "Clusters",
    resolution = 0.8
)

proj1 <- addUMAP(
    ArchRProj = proj1, 
    reducedDims = "IterativeLSI", 
    name = "UMAP", 
    nNeighbors = 30, 
    minDist = 0.5, 
    metric = "cosine"
)

HDF5_USE_FILE_LOCKING=FALSE
RHDF5_USE_FILE_LOCKING=FALSE

proj2 <- addGroupCoverages(ArchRProj = proj1, groupBy = "Clusters")

pathToMacs2 <- findMacs2()

proj2 <- addReproduciblePeakSet(
    ArchRProj = proj2, 
    groupBy = "Clusters", 
    pathToMacs2 = pathToMacs2
)


#saveArchRProject(ArchRProj = proj2, outputDirectory = "Save-Proj2_afterPeakCalling", load = FALSE)

proj3 <- addPeakMatrix(proj2)

peakMatrix <- getMatrixFromProject(proj3, useMatrix='PeakMatrix')

PeakMatrix_proj3_dense =  as.matrix(peakMatrix@assays@data@listData$PeakMatrix)

write.csv(PeakMatrix_proj3_dense, file = paste(out_path,"PeakMatrix_dense.csv",sep="" )) 

# get the peak regions
proj3_peak_metadata  = (getPeakSet(proj3))
proj3_peak_regions = data.frame(seqnames = seqnames(proj3_peak_metadata),
           start= start(proj3_peak_metadata),
            end = end(proj3_peak_metadata))

write.csv(proj3_peak_metadata, file =paste(out_path,"peak_metadata.csv",sep="" ))
write.csv(proj3_peak_regions, file =paste(out_path,"peak_regions.csv",sep="" ))
write.table(proj3_peak_regions, file =paste(out_path,"peak_regions.tsv",sep="" ),col.names = F,row.names=FALSE,sep="\t", quote = FALSE)

# save the peak matrix as sparse mtx


# save sparse matrix
sparse_matrix <- Matrix(PeakMatrix_proj3_dense , sparse = T )
writeMM(obj = sparse_matrix, file = paste(out_path,"ArchR_peakMatrix.mtx",sep="" ))

# save genes and cells names
combined_peak_features =  paste(proj3_peak_regions$seqnames,"_",proj3_peak_regions$start,"_",proj3_peak_regions$end, sep="")

write.table(as.data.frame(combined_peak_features), file = paste(out_path,"ArchR_peaks.tsv",sep="" ) , row.names = FALSE, quote = FALSE, col.names = FALSE)


Checking Inputs...

ArchR logging to : ArchRLogs/ArchR-addIterativeLSI-b94d6b0677e8-Date-2024-05-12_Time-23-13-09.log
If there is an issue, please report to github with logFile!

2024-05-12 23:13:11 : Computing Total Across All Features, 0.001 mins elapsed.

2024-05-12 23:13:12 : Computing Top Features, 0.017 mins elapsed.

###########
2024-05-12 23:13:14 : Running LSI (1 of 2) on Top Features, 0.048 mins elapsed.
###########

2024-05-12 23:13:14 : Sampling Cells (N = 10000) for Estimated LSI, 0.049 mins elapsed.

2024-05-12 23:13:14 : Creating Sampled Partial Matrix, 0.049 mins elapsed.

2024-05-12 23:13:36 : Computing Estimated LSI (projectAll = FALSE), 0.411 mins elapsed.

2024-05-12 23:14:53 : Identifying Clusters, 1.694 mins elapsed.

2024-05-12 23:15:25 : Identified 9 Clusters, 2.233 mins elapsed.

2024-05-12 23:15:25 : Saving LSI Iteration, 2.235 mins elapsed.

2024-05-12 23:15:58 : Creating Cluster Matrix on the total Group Features, 2.777 mins elapsed.

2024-05-12 23:17:03 : C

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 13745
Number of edges: 549245

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8934
Number of communities: 27
Elapsed time: 4 seconds


2024-05-12 23:18:54 : Testing Biased Clusters, 0.365 mins elapsed.

2024-05-12 23:18:55 : Testing Outlier Clusters, 0.372 mins elapsed.

2024-05-12 23:18:55 : Identified more clusters than maxClusters allowed (n = 0). Merging clusters to maxClusters (n = 25).
If this is not desired set maxClusters = NULL!, 0.373 mins elapsed.

2024-05-12 23:18:55 : Assigning Cluster Names to 25 Clusters, 0.373 mins elapsed.

2024-05-12 23:18:55 : Finished addClusters, 0.375 mins elapsed.

23:18:55 UMAP embedding parameters a = 0.583 b = 1.334

23:18:55 Read 13745 rows and found 30 numeric columns

23:18:55 Using Annoy for neighbor search, n_neighbors = 30

23:18:55 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

23:18:58 Writing NN index file to temp file /scratch/local/Rtmpl78HX8/file

    Group nCells nCellsUsed nReplicates nMin nMax maxPeaks
C1     C1     92         92           2   40   52    46000
C2     C2     99         99           2   40   59    49500
C3     C3    997        540           2   40  500   150000
C4     C4   2911        540           2   40  500   150000
C5     C5    137        137           2   40   97    68500
C6     C6    307        307           2   40  267   150000
C7     C7    445        445           2   40  405   150000
C8     C8     67         61           2   40   40    30500
C9     C9    383        383           2   40  343   150000
C10   C10    535        535           2   40  495   150000
C11   C11     53         52           2   40   40    26000
C12   C12   3461        540           2   40  500   150000
C13   C13    164        164           2   40  124    82000
C14   C14    397        397           2   40  357   150000
C15   C15    230        230           2   40  190   115000
C16   C16    486        486           2   40  446   1500

2024-05-13 00:31:22 : Batching Peak Calls!, 0.052 mins elapsed.

2024-05-13 00:31:22 : Batch Execution w/ safelapply!, 0 mins elapsed.

2024-05-13 00:31:22 : Group 1 of 48, Calling Peaks with MACS2!, 0 mins elapsed.

Running Macs2 with Params : macs2 callpeak -g 2.7e+09 --name C1._.Rep1-1 --treatment /project/scATAC_analysis/scATAcat_review/preprocessing/Granja2019/BMMC_D6T1/05_get_peaks_and_genesScore_matrix_ArchR/BMMC_D6T1/PeakCalls/InsertionBeds/C1._.Rep1-1.insertions.bed --outdir /project/scATAC_analysis/scATAcat_review/preprocessing/Granja2019/BMMC_D6T1/05_get_peaks_and_genesScore_matrix_ArchR/BMMC_D6T1/PeakCalls/InsertionBeds --format BED --call-summits --keep-dup all --nomodel --nolambda --shift -75 --extsize 150 -q 0.1

2024-05-13 00:31:33 : Group 2 of 48, Calling Peaks with MACS2!, 0.173 mins elapsed.

Running Macs2 with Params : macs2 callpeak -g 2.7e+09 --name C1._.Rep2-2 --treatment /project/scATAC_analysis/scATAcat_review/preprocessing/Granja2019/BMMC_D6T1/05_get_peaks_and

...

[1] "/project/scATAC_analysis/scATAcat_review/preprocessing/Granja2019/BMMC_D6T1/05_get_peaks_and_genesScore_matrix_ArchR/BMMC_D6T1/PeakCalls/C8-reproduciblePeaks.gr.rds"


Annotating Peaks : Nearest Gene

Annotating Peaks : Gene

Annotating Peaks : TSS

Annotating Peaks : GC



[1] "/project/scATAC_analysis/scATAcat_review/preprocessing/Granja2019/BMMC_D6T1/05_get_peaks_and_genesScore_matrix_ArchR/BMMC_D6T1/PeakCalls/C9-reproduciblePeaks.gr.rds"


2024-05-13 01:24:57 : Creating Union Peak Set!, 53.638 mins elapsed.

Converged after 8 iterations!

Plotting Ggplot!

2024-05-13 01:25:06 : Finished Creating Union Peak Set (198362)!, 53.794 mins elapsed.

ArchR logging to : ArchRLogs/ArchR-addPeakMatrix-b94d53288b59-Date-2024-05-13_Time-01-25-06.log
If there is an issue, please report to github with logFile!

2024-05-13 01:25:07 : Batch Execution w/ safelapply!, 0 mins elapsed.

2024-05-13 01:25:09 : Adding BMMC_D6T1 to PeakMatrix for Chr (1 of 23)!, 0.024 mins elapsed.

2024-05-13 01:25:26 : Adding BMMC_D6T1 to PeakMatrix for Chr (2 of 23)!, 0.314 mins elapsed.

2024-05-13 01:25:39 : Adding BMMC_D6T1 to PeakMatrix for Chr (3 of 23)!, 0.531 mins elapsed.

2024-05-13 01:25:49 : Adding BMMC_D6T1 to PeakMatrix for Chr (4 of 23)!, 0.699 mins elapsed.

2024-05-13 01:25:57 : Adding BMMC_D6T1 to PeakMatrix for Chr (5 of 23)!, 0.83 mins elapsed.

2024-05-13 01:26:06 : Adding BMMC_D6T1 to PeakMatrix for Chr (6 of 23)!, 0.979 mins elapsed.

20

NULL

In [18]:
proj1


           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /project/scATAC_analysis/scATAcat_review/preprocessing/Granja2019/BMMC_D6T1/05_get_peaks_and_genesScore_matrix_ArchR/BMMC_D6T1 
samples(1): BMMC_D6T1
sampleColData names(1): ArrowFiles
cellColData names(14): Sample nMultiFrags ... BlacklistRatio Clusters
numberOfCells(1): 13745
medianTSS(1): 14.317
medianFrags(1): 9853

In [19]:
getAvailableMatrices(proj3)

[1] "GeneScoreMatrix" "PeakMatrix"      "TileMatrix"

In [20]:
# get the gene activity matrix 

geneScoreMatrix <- getMatrixFromProject(proj3, useMatrix='GeneScoreMatrix')




ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-b94d155fe190-Date-2024-05-13_Time-02-08-45.log
If there is an issue, please report to github with logFile!

2024-05-13 02:10:54 : Organizing colData, 2.16 mins elapsed.

2024-05-13 02:10:55 : Organizing rowData, 2.178 mins elapsed.

2024-05-13 02:10:58 : Organizing rowRanges, 2.218 mins elapsed.

2024-05-13 02:10:59 : Organizing Assays (1 of 1), 2.236 mins elapsed.

2024-05-13 02:11:00 : Constructing SummarizedExperiment, 2.246 mins elapsed.

2024-05-13 02:11:02 : Finished Matrix Creation, 2.295 mins elapsed.



In [21]:
dim(getMatrixFromProject(proj3, useMatrix='PeakMatrix'))

ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-b94d256cc192-Date-2024-05-13_Time-02-11-04.log
If there is an issue, please report to github with logFile!

2024-05-13 02:13:14 : Organizing colData, 2.168 mins elapsed.

2024-05-13 02:13:15 : Organizing rowData, 2.183 mins elapsed.

2024-05-13 02:13:17 : Organizing rowRanges, 2.217 mins elapsed.

2024-05-13 02:13:18 : Organizing Assays (1 of 1), 2.236 mins elapsed.

2024-05-13 02:13:19 : Constructing SummarizedExperiment, 2.252 mins elapsed.

2024-05-13 02:13:23 : Finished Matrix Creation, 2.312 mins elapsed.



[1] 198362  13745

In [29]:
dim(geneScoreMatrix)

[1] 24919 13745

In [22]:
geneScore_outDir = "/project/scATAC_analysis/scATAcat_review/data/Granja2019/BMMC_D6T1/05_peaks_and_genesScore_matrix_ArchR/"

In [23]:
write.table(rowData(geneScoreMatrix)$name, paste(geneScore_outDir, "ArchR_genescore_genes.tsv", sep=""),quote = FALSE, row.names=FALSE, col.names = FALSE)

In [24]:
write.table(colnames(geneScoreMatrix), paste(geneScore_outDir, "ArchR_genescore_barcodes.tsv", sep=""),quote = FALSE, row.names=FALSE, col.names = FALSE)

In [25]:
geneScoreMatrix_sparse = geneScoreMatrix@assays@data@listData$GeneScoreMatrix


In [26]:
writeMM(obj = geneScoreMatrix_sparse, file=paste(geneScore_outDir, "ArchR_genescore.mtx", sep=""))

NULL

In [27]:
saveArchRProject(ArchRProj = proj3, outputDirectory = paste(geneScore_outDir,"Save-Proj", sep=""), load = FALSE)

Copying ArchRProject to new outputDirectory : /project/scATAC_analysis/scATAcat_review/data/Granja2019/BMMC_D6T1/05_peaks_and_genesScore_matrix_ArchR/Save-Proj

Copying Arrow Files...

Copying Arrow Files (1 of 1)

Getting ImputeWeights

No imputeWeights found, returning NULL

Copying Other Files...

Copying Other Files (1 of 5): Embeddings

Copying Other Files (2 of 5): GroupCoverages

Copying Other Files (3 of 5): IterativeLSI

Copying Other Files (4 of 5): PeakCalls

Copying Other Files (5 of 5): Plots

Saving ArchRProject...



In [28]:
library(stringr)

write.table(as.data.frame(str_split_fixed(colnames(geneScoreMatrix),'#',2))$V2, paste(geneScore_outDir, "cell_IDs.tsv", sep=""),quote = FALSE, row.names=FALSE, col.names = FALSE)
